### Prepare for TimeSliderChoropleth

In [ ]:
import branca.colormap as cm
# take log of import values in mil?bil? so that they are comparable between different sizes of countries' imports
gdf_commodity1['log_value'] = np.log10(gdf_commodity1['value']+1)
max_colour = max(gdf_commodity1['log_value'])
min_colour = min(gdf_commodity1['log_value'])
#cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
cmap = cm.linear.Reds_09.scale(min_colour, max_colour)
gdf_commodity1['colour'] = gdf_commodity1['log_value'].map(cmap)

ValueError: ignored

In [ ]:
# Create style_dict for the TimeSliderChoropleth (choice of color and opacity)
country_list = gdf_commodity1['COUNTRY'].unique().tolist()
country_idx = range(len(country_list))

style_dict = {}
for i in country_idx:
    country = country_list[i]
    result = gdf_commodity1[gdf_commodity1['COUNTRY'] == country]
    # Reorder data by country and date
    result = result.sort_values(by=['date'])
    # Reset index
    result.reset_index(inplace=True,drop=True)

    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[int(r['date_sec'])] = {'color': r['colour'], 'opacity': 0.7}
    style_dict[str(i)] = inner_dict

# Resources

**Tools**  
[Folium Bubble Map](https://python-graph-gallery.com/313-bubble-map-with-folium/)  
[Individual Choropleth Popups](http://www.mattgoldwasser.com/posts/choroplot/)  
[TimeSliderChoropleth Example](https://github.com/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb)  
[Time Series Folium Example](https://geohackweek.github.io/ghw2018_web_portal_inlandwater_co2/InteractiveTimeSeries.html)  


**Dataset**  
[Annual GDP Growth %](https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG)  
bot_with_tariffs_calcs.csv - Paul Sail generated this file based on exports & imports for UK  
[countries_lat_lon.csv](https://developers.google.com/public-data/docs/canonical/countries_csv)  




# Imports & Settings

In [ ]:
!pip install geopandas
!pip install ipyleaflet
!pip install beautifulsoup4
!pip install geopy
!pip install googlemaps
!pip install zipcodes
!pip install addfips #Finds fips code from county and state name

# Import Analysis Libraries
import pandas as pd
import numpy as np
from collections import Counter,defaultdict
import os

# Other Tools
from tqdm import trange,tqdm
from geopy.geocoders import Nominatim # find latitude and longitude by county names for folium mapping
import googlemaps # Geolocation finder
from datetime import datetime
import zipcodes
import addfips
from datetime import datetime
import numpy as np
import time
from collections import OrderedDict

# Visualization
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager
color_palette_list = ['#009ACD', '#ADD8E6', '#63D1F4', '#0EBFE9','#C1F0F6', '#0099CC']
import geopandas as gpd
from ipyleaflet import Map, GeoData, basemaps, LayersControl
%matplotlib inline
import folium
from folium.plugins import TimeSliderChoropleth

# Jupyter Notebook & Pandas Set
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
pd.set_option('display.max_columns',100)

# WebScraping
import requests
from bs4 import BeautifulSoup

# ETC
import warnings
warnings.filterwarnings('ignore')

# Set up connectivity with gdrive with colab
from google.colab import drive
drive.mount('/content/gdrive')

# Functions

## Folium Functions

In [2]:
# help(folium.FeatureGroup)
def folium_census(mapobj,gdf,key,color,show=False):
  # Prep data
  folium_gdf = gdf
  folium_gdf.crs = {'init' :'epsg:4326'}
  json_folium = folium_gdf.to_crs(epsg='4326').to_json()

  folium.Choropleth(
  geo_data=json_folium,
  name="COVID-19 Confirmed Cases Total",
  data=folium_gdf,
  columns=['GEOID', key],
  key_on='properties.GEOID',
  fill_color=color, # All colors : http://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3
  fill_opacity=1,
  line_opacity=3,
  legend_name=key,
  show=show
  ).add_to(mapobj)

  return mapobj

def folium_marker(mapobj,popup_msg,icon_color,icon_type,longitude,latitude):
  # https://getbootstrap.com/docs/3.3/components/#glyphicons-glyphs for built-in available icon types
  folium.Marker(
    location=[latitude,longitude],
    popup=popup_msg,
    icon=folium.Icon(color=icon_color,icon=icon_type)
  ).add_to(mapobj)
  return mapobj


def folium_marker_cluster(mapobj,df,cluster_name,popup_msg,icon_color,icon_type,show=False):

  from folium.plugins import MarkerCluster
  icon_create_function = """
      function(cluster) {
      var childCount = cluster.getChildCount(); 
      var c = ' marker-cluster-';

      if (childCount < 200) {
          c += 'large';
      } else if (childCount < 500) {
          c += 'medium';
      } else {
          c += 'small';
      }

      return new L.DivIcon({ html: '<div><span>' + childCount + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) });
      }
      """
  mc = MarkerCluster(name=cluster_name,icon_create_function=icon_create_function,show=show)
  #mc = MarkerCluster(name=cluster_name,show=False)
  mc.add_to(mapobj)

  for i in range(len(df)):
    mc.add_child(folium.Marker(location = [df['Latitude'].iloc[i],df['Longitude'].iloc[i]],
                  popup=popup_msg,
                  icon=folium.Icon(color=icon_color,icon=icon_type)
                  ))
  return mapobj

def folium_feature_group(mapobj,df,cluster_name,icon_color,icon_type,show=False):
  '''
  Below blocked out code lets urls to be included, couldn't get it to work yet.
  '''
  # def get_frame(url,width,height):
  #   html = """ 
  #           <!doctype html>
  #       <html>
  #       <iframe id="myIFrame" width="{}" height="{}" src={}""".format(width,height,url) + """ frameborder="0"></iframe>
  #       <script type="text/javascript">
  #       var resizeIFrame = function(event) {
  #           var loc = document.location;
  #           if (event.origin != loc.protocol + '//' + loc.host) return;

  #           var myIFrame = document.getElementById('myIFrame');
  #           if (myIFrame) {
  #               myIFrame.style.height = event.data.h + "px";
  #               myIFrame.style.width  = event.data.w + "px";
  #           }
  #       };
  #       if (window.addEventListener) {
  #           window.addEventListener("message", resizeIFrame, false);
  #       } else if (window.attachEvent) {
  #           window.attachEvent("onmessage", resizeIFrame);
  #       }
  #       </script>
  #       </html>"""
  #   return html
  #   popup = get_frame('https://stackoverflow.com/questions/29535715/python-with-folium-how-can-i-embed-a-webpage-in-the-popup',
  #             500,
  #             500)



  fg = folium.FeatureGroup(name=cluster_name,show=show)

  print("Populating Annual GDP Growth graph per country...")
  time.sleep(1)
  for i in trange(len(df)):

    import base64
    import matplotlib.pyplot as plt

    resolution, width, height = 75, 15, 4

    fig, ax = plt.subplots(figsize=(width, height))
    ax.plot( gdp_growth[gdp_growth['COUNTRY']==df['country'].iloc[i]]['year'], gdp_growth[gdp_growth['COUNTRY']==df['country'].iloc[i]]['value'])
    plt.xticks(rotation=80)
    plt.ylabel('GDP Growth %')
    ax.set_title(df['country'].iloc[i])
    plt.close(fig)
    png = 'viz.png'

    fig.savefig(png, dpi=resolution)

    encoded = base64.b64encode(open(png, 'rb').read())

    from folium import IFrame

    html = '<img src="data:image/png;base64,{}">'.format
    iframe = IFrame(html(encoded.decode('UTF-8')), width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe, max_width=2650)



    fg.add_child(folium.Marker(location = [df['latitude'].iloc[i],df['longitude'].iloc[i]],
                  popup=popup,
                  icon=folium.Icon(color=icon_color,icon=icon_type)
                  ))
    
  fg.add_to(mapobj)
  return mapobj



# Prepare Dataset

### UK Imports Data

In [3]:
print("Importing balance of trade Data...")

uk_bot = pd.read_csv('gdrive/My Drive/PA Consulting/DIT EU Support/bot_with_tariffs_calcs_updated.csv') 

# Create a new column with country abbreviations removed. ex) "AE United Arab Emirates" -> "United Arab Emirates" to merge it with shapefiles
uk_bot['COUNTRY'] = uk_bot['COUNTRY'].apply( lambda x :  ' '.join(x.split()[1:]) )

print("Done")

Importing balance of trade Data...
Done


In [4]:
# Rename some countries' name to be able to merge with shapefiles

uk_bot = uk_bot.replace({'COUNTRY' : 
                      dict.fromkeys(['Hong Kong', 
                                     'Macau'], 
                                     'China')})

uk_bot = uk_bot.replace({'COUNTRY' : 
                      dict.fromkeys(['Vatican City'], 
                                     'Italy')})


### Import lon & lat data for each country for markers

In [5]:
country_markers = pd.read_csv('gdrive/My Drive/PA Consulting/DIT EU Support/countries_lat_lon.csv', encoding = "ISO-8859-1") 
country_markers = country_markers[country_markers['country']!='U.S. Minor Outlying Islands']

### Annual GDP Growth % data for marker's popup messages

In [6]:
gdp_growth = pd.read_csv('gdrive/My Drive/PA Consulting/DIT EU Support/gdp_annual_percentage.csv',skiprows=4)  #https://data.worldbank.org/indicator/NY.GDP.MKTP.KD.ZG
gdp_growth = gdp_growth.rename(columns={'Country Name': 'COUNTRY'})
gdp_growth.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code','Unnamed: 64'],inplace=True)

# Stretch out the data
gdp_growth = gdp_growth.melt(id_vars=['COUNTRY'], var_name='year')

# Drop na and re order data by country and year
gdp_growth.dropna(inplace=True)
gdp_growth = gdp_growth.sort_values(by=['COUNTRY','year'])

### Country Shape Files

In [7]:
print("Loading country shape files for Folium...")
gdf_countries = gpd.GeoDataFrame.from_file('gdrive/My Drive/shapefiles/world countries/99bfd9e7-bb42-4728-87b5-07f8c8ac631c2020328-1-1vef4ev.lu5nk.shp') #https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_county_500k.zip <- All Counties in US
gdf_countries = gdf_countries.rename(columns={'CNTRY_NAME': 'COUNTRY'})
gdf_countries.drop(columns=['OBJECTID'],inplace=True)
print("Done")

Loading country shape files for Folium...
Done


### Data Inspection

In [8]:
# Not all countries in import & export files exist in shape files, so those won't be displayed on Folium
countries_in_shape_files = []
countries_not_in_shape_files = []

for country in list(set(uk_bot['COUNTRY'])):
  if country not in list(set(uk_bot['COUNTRY']) & set(gdf_countries['COUNTRY'])):
    countries_not_in_shape_files.append(country)
  else:
    countries_in_shape_files.append(country)
countries_not_in_shape_files = sorted(countries_not_in_shape_files)
countries_in_shape_files = sorted(countries_in_shape_files)


In [9]:
# Unique commodities
commodities = list(uk_bot['COMMODITY'].unique())

### Merge data with shape files

In [ ]:
print("Merging all data...")

gdf_commodity1 = uk_bot_commodity1.merge(gdf_countries, left_on='COUNTRY',right_on='COUNTRY')

print("Done")

Merging all data...
Done


In [13]:
# Create geopandas dataframe
countries_df = gdf_commodity1[['geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
countries_gdf = countries_gdf.drop_duplicates().reset_index()

### Function to select subset of data by commodity and prepare data for TimeSliderChoropleth

In [18]:
def timesliderchoropleth_oneshot(commodity):
  uk_imports_commodity1 = uk_bot[uk_bot['COMMODITY'] == commodity]

  # Drop unneeded columns
  uk_imports_commodity1.drop(columns=['DIRECTION','commodity_key'],inplace=True)

  # Stretch out the data
  uk_imports_commodity1 = uk_imports_commodity1.melt(id_vars=['COMMODITY', 'COUNTRY'], var_name='date')

  # Reformat date to actual datetime
  uk_imports_commodity1['date'] = uk_imports_commodity1['date'].apply(lambda x:datetime.strptime(x.split('_')[0],'%Y%b'))
  uk_imports_commodity1['date_sec'] = uk_imports_commodity1['date'].astype(int) / 10**9
  uk_imports_commodity1['date_sec'] = uk_imports_commodity1['date_sec'].astype(int).astype(str)

  # Reorder data by country and date
  uk_imports_commodity1 = uk_imports_commodity1.sort_values(by=['COUNTRY','date'])

  # Reset index
  uk_imports_commodity1.reset_index(inplace=True,drop=True)

  gdf_commodity1 = uk_imports_commodity1.merge(gdf_countries, left_on='COUNTRY',right_on='COUNTRY')

  import branca.colormap as cm


  gdf_commodity1.loc[gdf_commodity1['date']=='1998-01-01','value'] = 0
  gdf_commodity1.dropna(inplace=True)

  max_colour = max(gdf_commodity1['value'])
  min_colour = min(gdf_commodity1['value'])
  #cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
  #cmap = cm.linear.Reds_09.scale(min_colour, max_colour)
  cmap = cm.LinearColormap( colors=['red','green'], vmin=min_colour,vmax=max_colour)
  gdf_commodity1['colour'] = gdf_commodity1['value'].map(cmap)

  # Create style_dict for the TimeSliderChoropleth (choice of color and opacity)
  country_list = gdf_commodity1['COUNTRY'].unique().tolist()
  country_idx = range(len(country_list))

  style_dict = OrderedDict()
  for i in country_idx:
      country = country_list[i]
      result = gdf_commodity1[gdf_commodity1['COUNTRY'] == country]

      # Reorder data by country and date
      result = result.sort_values(by=['date'])
      # Reset index
      result.reset_index(inplace=True,drop=True)

      inner_dict = {}
      for _, r in result.iterrows():
        if r['value']==0:
          inner_dict[int(r['date_sec'])] = {'color': r['colour'], 'opacity': 0}
        else:
          inner_dict[int(r['date_sec'])] = {'color': r['colour'], 'opacity': 0.4}
      style_dict[str(i)] = inner_dict

  for key in style_dict.keys():
    style_dict[key][883612800]['opacity'] = 0

  # Create geopandas dataframe
  countries_df = gdf_commodity1[['geometry']]
  countries_gdf = gpd.GeoDataFrame(countries_df)
  countries_gdf = countries_gdf.drop_duplicates().reset_index()
  return countries_gdf,style_dict,cmap

# Folium

### Marker popup GDP growth

In [ ]:
# Initiate map
m = folium.Map(min_zoom=2, max_bounds=True,tiles='cartodbpositron')

# Add commodities timesliderchoropleth
print("Generating commodities timesliderchoropleth for all countries...")
time.sleep(1)

commodities_list = ['0 Food & live animals','3 Fuels','5 Chemicals']
for i in trange(len(commodities_list)):
  countries_gdf,style_dict,cmap =  timesliderchoropleth_oneshot(commodities_list[i])
  g = TimeSliderChoropleth(    
      data=countries_gdf.to_json(),
      styledict=style_dict,name='Balance of Trade : ' + ' '.join(commodities_list[i].split()[1:]),
      overlay=True,
      control=True,
      show=True).add_to(m)
  cmap.caption = 'Balance of Trade : ' + ' '.join(commodities_list[i].split()[1:]) + ' in £ (mil)'
  cmap.add_to(m)

# Create markers for each country with annual gdp growth % graph
m = folium_feature_group(mapobj=m,
                          df=country_markers,
                          cluster_name='Annual GDP Growth %',
                          icon_color='black',
                          icon_type='info-sign',
                          show=False)

folium.LayerControl().add_to(m)

m.save(outfile='gdrive/My Drive/PA Consulting/DIT EU Support/DIT demo v1.html')
m

In [21]:
rest_of_commodities = ['00 Live animals',
 '01 Meat & meat preparations',
 '02 Dairy products & eggs',
 '03 Fish & shellfish',
 '04 Cereals',
 '05 Vegetables & fruit',
 '06 Sugar',
 '07 Coffee, tea, cocoa etc',
 '08 Animal feeding stuffs',
 '09 Miscellaneous foods',
 '32 Coal, coke & briquettes',
 '33 Oil',
 '33O Crude oil',
 '33R Refined oil',
 '34 Gas',
 '35 Electricity',
 '3OF Fuels other than oil',
 '51 Organic chemicals',
 '52 Inorganic chemicals',
 '53 Dyeing, tanning & colouring materials',
 '54 Medicinal & pharmaceutical products',
 '55 Toilet & cleansing preparations',
 '56 Processed fertilisers',
 '57 Plastics in primary forms',
 '58 Plastics in non-primary forms',
 '59 Other chemicals']

In [ ]:
for k in range(9):
  

  # Initiate map
  m = folium.Map(min_zoom=2, max_bounds=True,tiles='cartodbpositron')

  # Add commodities timesliderchoropleth
  print("Generating commodities timesliderchoropleth for all countries...")
  time.sleep(1)

  commodities_list = rest_of_commodities[k*3:k*3+3]
  for i in trange(len(commodities_list)):
    countries_gdf,style_dict,cmap =  timesliderchoropleth_oneshot(commodities_list[i])
    g = TimeSliderChoropleth(    
        data=countries_gdf.to_json(),
        styledict=style_dict,name='Balance of Trade : ' + ' '.join(commodities_list[i].split()[1:]),
        overlay=True,
        control=True,
        show=True).add_to(m)
    cmap.caption = 'Balance of Trade : ' + ' '.join(commodities_list[i].split()[1:]) + ' in £ (mil)'
    cmap.add_to(m)

  # Create markers for each country with annual gdp growth % graph
  m = folium_feature_group(mapobj=m,
                            df=country_markers,
                            cluster_name='Annual GDP Growth %',
                            icon_color='black',
                            icon_type='info-sign',
                            show=False)

  folium.LayerControl().add_to(m)

  m.save(outfile='gdrive/My Drive/PA Consulting/DIT EU Support/DIT demo v{}.html'.format(k))
  m

Generating commodities timesliderchoropleth for all countries...


100%|██████████| 3/3 [02:08<00:00, 42.70s/it]


Populating Annual GDP Growth graph per country...


100%|██████████| 244/244 [00:48<00:00,  5.01it/s]


Generating commodities timesliderchoropleth for all countries...


 67%|██████▋   | 2/3 [01:23<00:42, 42.06s/it]